# Retrieval Augmented Generation - RAG 

We will use the RAG technique to use language models to attempt to solve the multi-class, multi-label classification problem.

In [2]:
from utils import parse_sgm_to_dataframe

## Data Exploration

In [3]:
df = parse_sgm_to_dataframe('../../data/reuters21578/reut2-000.sgm')

In [4]:
def check_topics_in_file(topic_list, file_path):
    # Read topics from the file
    with open(file_path, 'r') as file:
        file_topics = set(file.read().splitlines())
    
    # Check if all topics in topic_list are in file_topics
    missing_topics = set(topic_list) - file_topics
    
    if not missing_topics:
        return True, []
    else:
        return False, list(missing_topics)

In [5]:
topic_list = df['Topic'].unique()

In [7]:
check_topics_in_file(topic_list, '../../data/reuters21578/all-topics-strings.lc.txt')

(False, ['', 'cotton'])

In [8]:
topic_file = '../../data/reuters21578/all-topics-strings.lc.txt'
with open(topic_file, 'r') as file:
    file_topics = set(file.read().splitlines())

In [9]:
topics = list(file_topics)

In [10]:
topics = [s.strip() for s in topics]

In [11]:
topics.sort()

In [13]:
df.head()

,ID,Topic,Body
0,1,cocoa,Showers continued throughout the week in\nthe ...
1,2,,Standard Oil Co and BP North America\nInc said...
2,3,,Texas Commerce Bancshares Inc's Texas\nCommerc...
3,4,,BankAmerica Corp is not under\npressure to act...
4,5,grain,The U.S. Agriculture Department\nreported the ...


## Find suitable pre-trained models

## Explore the use of models pretrained on article data

## Optimize our RAG pipeline

## Explore retrieval improvements

Retrieval is in-expensive so it is in most cases a good tradeoff to do more on the retrieval side in order to ensure that we have a good list of candidate labels that includes the true labels.

To explore:
- Sentence splitting
- Retrieval ensemble